In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rectal_Cancer/GSE150082'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Pre-existing tumoral B cell infiltration and impaired genome maintenance correlate with response to chemoradiotherapy in locally advanced rectal cancer (LARC)"
!Series_summary	"Using Human Genome 4x44 two-color Agilent microarrays, we established the expression profiling of 39 LARC pretreatment tumor samples to elucidate the molecular features associated with response to treatment after neoadjuvant chemoradiotherapy (nCRT)."
!Series_overall_design	"Two color microarrays where Cy5= tumor sample and Cy3= Stratagene Universal Human RNA Reference. This dataset comprises the transcriptomic profiling of 39 consecutive eligible LARC patients who underwent therapy at the Oncology Unit at Bonorino Udaondo Hospital (Buenos Aires, Argentina) from November 2015 to September 2018. This study was approved by the Udaondo Hospital Ethics Committee and the Instituto Leloir Institutional Review Board. All patients signed the approved Informed Consent.  All patients

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True

# Identify the keys in the sample characteristics dictionary where unique values are recorded
# For Rectal_Cancer trait (using ptrg values to infer Rectal_Cancer association)
trait_row = 3

# For age values
age_row = 2

# For gender values
gender_row = 0

# Define the data type conversion functions
def convert_trait(value):
    mapping = {
        'Complete_clinical_response_nonOperative': 1,
        '0': 1,
        '1': 1,
        '2': 0,
        '3': 0,
        'Unresectable': 0,
        'NA': None
    }
    key_value = value.split(':')[-1].strip()
    return mapping.get(key_value, None)

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    mapping = {
        'M': 1,
        'F': 0
    }
    key_value = value.split(':')[-1].strip()
    return mapping.get(key_value, None)

# Save cohort information
save_cohort_info('GSE150082', './preprocessed/Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract relevant features
if trait_row is not None and age_row is not None and gender_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rectal_Cancer/trait_data/GSE150082.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4523129': [1.0, 70.0, 1.0], 'GSM4523130': [1.0, 74.0, 1.0], 'GSM4523131': [nan, 45.0, 0.0], 'GSM4523132': [1.0, 45.0, 0.0], 'GSM4523133': [1.0, 54.0, 1.0], 'GSM4523134': [1.0, 72.0, 1.0], 'GSM4523135': [1.0, 57.0, 1.0], 'GSM4523136': [1.0, 66.0, 1.0], 'GSM4523137': [1.0, 71.0, 0.0], 'GSM4523138': [1.0, 47.0, 1.0], 'GSM4523139': [1.0, 66.0, 1.0], 'GSM4523140': [1.0, 61.0, 1.0], 'GSM4523141': [1.0, 64.0, 1.0], 'GSM4523142': [1.0, 61.0, 1.0], 'GSM4523143': [1.0, 59.0, 1.0], 'GSM4523144': [0.0, 34.0, 0.0], 'GSM4523145': [0.0, 63.0, 1.0], 'GSM4523146': [0.0, 66.0, 0.0], 'GSM4523147': [0.0, 46.0, 1.0], 'GSM4523148': [0.0, 57.0, 1.0], 'GSM4523149': [0.0, 64.0, 1.0], 'GSM4523150': [0.0, 55.0, 1.0], 'GSM4523151': [0.0, 75.0, 1.0], 'GSM4523152': [0.0, 61.0, 1.0], 'GSM4523153': [0.0, 42.0, 0.0], 'GSM4523154': [0.0, 63.0, 1.0], 'GSM4523155': [0.0, 42.0, 1.0], 'GSM4523156': [0.0, 69.0, 1.0], 'GSM4523157': [0.0, 49.0, 0.0], 'GSM4523158': [0.0, 68.0, 0.0], 'GSM4523159': [0.0, 60.0, 0.0], 'GSM452

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Rectal_Cancer/gene_data/GSE150082.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Rectal_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE150082', './preprocessed/Rectal_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Rectal_Cancer/GSE150082.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Rectal_Cancer', the least common label is '1.0' with 15 occurrences. This represents 39.47% of the dataset.
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.25
  50% (Median): 61.0
  75%: 66.0
Min: 30.0
Max: 75.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 23.68% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

